In [1]:
import pandas as pd
import numpy as np

import requests
import warnings
warnings.filterwarnings('ignore')
import json
from pprint import pprint


from os import path

import matplotlib
import matplotlib.pyplot as plt
plt.rc('font', size='14')

from datetime import timedelta, datetime

from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
import statsmodels.api as sm
from statsmodels.tsa.ar_model import AR

from sklearn.model_selection import TimeSeriesSplit
from sklearn import metrics

import math

import model
import prepare
import explore

In [39]:
df = pd.read_csv('activity.csv')

In [40]:
df = prepare.prep_df(df)

In [41]:
train, test = model.train_test_split(df)

In [42]:
test.tail()

,Calories Burned,Steps,Distance,Floors,Minutes Sedentary,Minutes Lightly Active,Minutes Fairly Active,Minutes Very Active,Activity Calories
Date,,,,,,,,,
2018-12-02,3957,13257,6.23,195,725,265,13,70,2305
2018-12-03,3660,12915,6.12,6,782,219,40,51,1993
2018-12-04,3384,13763,6.49,13,608,199,11,67,1663
2018-12-05,3670,13865,6.52,12,739,200,12,69,1968
2018-12-06,3669,14774,6.96,9,647,198,4,78,1962


In [43]:
yhat = pd.DataFrame(test)
yhat.reset_index(inplace = True)

In [44]:
str = '2018-12-07'

In [45]:
x = datetime.strptime(str, '%Y-%m-%d')
print(x)

2018-12-07 00:00:00


In [46]:
for r in range(0, len(yhat.Date)):
    yhat.Date[r] = x + timedelta(r)
yhat.Date

0    2018-12-07
1    2018-12-08
2    2018-12-09
3    2018-12-10
4    2018-12-11
5    2018-12-12
6    2018-12-13
7    2018-12-14
8    2018-12-15
9    2018-12-16
10   2018-12-17
11   2018-12-18
12   2018-12-19
13   2018-12-20
Name: Date, dtype: datetime64[ns]

In [47]:
yhat.set_index('Date',inplace = True)


In [30]:
yhat = yhat[['Calories Burned']]
yhat

,Calories Burned
Date,
2018-12-07,3325
2018-12-08,4098
2018-12-09,3005
2018-12-10,3364
2018-12-11,3726
2018-12-12,3088
2018-12-13,3693
2018-12-14,3269
2018-12-15,3813


In [32]:
train = df[['Calories Burned']]
train

,Calories Burned
Date,
2018-04-26,2635
2018-04-27,3649
2018-04-28,3379
2018-04-29,2851
2018-04-30,2825
2018-05-01,3292
2018-05-02,2660
2018-05-03,3121
2018-05-04,2522


In [33]:
test = yhat
test

,Calories Burned
Date,
2018-12-07,3325
2018-12-08,4098
2018-12-09,3005
2018-12-10,3364
2018-12-11,3726
2018-12-12,3088
2018-12-13,3693
2018-12-14,3269
2018-12-15,3813


In [34]:
def holt_linear_trend(yhat, train, test):
    for var in train.columns:
        models = Holt(train[var]).fit(smoothing_level=.3, smoothing_slope=.1, optimized=False)
        yhat[var] = pd.DataFrame(models.forecast(test[var].shape[0]), columns=[var])
    return yhat

In [36]:
yhat['Calories Burned'] = holt_linear_trend(yhat,train,test)

In [48]:
for i in train.columns:
    yhat_var = yhat[[i]]
    train = df[[i]]
    test = yhat_var
    yhat[i] = holt_linear_trend(yhat_var,train,test)


In [49]:
yhat

,Calories Burned,Steps,Distance,Floors,Minutes Sedentary,Minutes Lightly Active,Minutes Fairly Active,Minutes Very Active,Activity Calories
Date,,,,,,,,,
2018-12-07,3587.472100,13585.304102,6.435701,25.337062,694.307013,208.806811,12.007653,64.611709,1887.870661
2018-12-08,3580.626709,13671.018509,6.462914,24.637360,698.961113,205.429933,11.587490,65.558456,1879.472303
2018-12-09,3573.781319,13756.732916,6.490128,23.937657,703.615214,202.053055,11.167326,66.505202,1871.073946
2018-12-10,3566.935928,13842.447323,6.517341,23.237955,708.269315,198.676177,10.747163,67.451949,1862.675589
2018-12-11,3560.090537,13928.161730,6.544555,22.538252,712.923416,195.299298,10.326999,68.398695,1854.277232
2018-12-12,3553.245146,14013.876137,6.571768,21.838550,717.577516,191.922420,9.906836,69.345442,1845.878875
2018-12-13,3546.399755,14099.590544,6.598981,21.138847,722.231617,188.545542,9.486672,70.292189,1837.480517
2018-12-14,3539.554365,14185.304951,6.626195,20.439145,726.885718,185.168664,9.066509,71.238935,1829.082160
2018-12-15,3532.708974,14271.019359,6.653408,19.739442,731.539819,181.791786,8.646345,72.185682,1820.683803


In [52]:
train = df
train.columns

Index(['Calories Burned', 'Steps', 'Distance', 'Floors', 'Minutes Sedentary',
       'Minutes Lightly Active', 'Minutes Fairly Active',
       'Minutes Very Active', 'Activity Calories'],
      dtype='object')

In [53]:
model1 = Holt(train['Calories Burned'],exponential =True).fit(smoothing_level=.3, smoothing_slope=.1, optimized=False)
model5 = Holt(train['Minutes Sedentary'],exponential =True).fit(smoothing_level=.3, smoothing_slope=.1, optimized=False)

In [55]:
yhat['Calories Burned'] = pd.DataFrame(model1.forecast(yhat['Calories Burned'].shape[0]), columns = ['Calories Burned'])
yhat['Minutes Sedentary'] = pd.DataFrame(model5.forecast(yhat['Minutes Sedentary'].shape[0]), columns = ['Minutes Sedentary'])

In [56]:
yhat

,Calories Burned,Steps,Distance,Floors,Minutes Sedentary,Minutes Lightly Active,Minutes Fairly Active,Minutes Very Active,Activity Calories
Date,,,,,,,,,
2018-12-07,3597.516452,13585.304102,6.435701,25.337062,698.298997,208.806811,12.007653,64.611709,1887.870661
2018-12-08,3592.843220,13671.018509,6.462914,24.637360,704.073280,205.429933,11.587490,65.558456,1879.472303
2018-12-09,3588.176057,13756.732916,6.490128,23.937657,709.895310,202.053055,11.167326,66.505202,1871.073946
2018-12-10,3583.514958,13842.447323,6.517341,23.237955,715.765483,198.676177,10.747163,67.451949,1862.675589
2018-12-11,3578.859913,13928.161730,6.544555,22.538252,721.684197,195.299298,10.326999,68.398695,1854.277232
2018-12-12,3574.210916,14013.876137,6.571768,21.838550,727.651853,191.922420,9.906836,69.345442,1845.878875
2018-12-13,3569.567957,14099.590544,6.598981,21.138847,733.668856,188.545542,9.486672,70.292189,1837.480517
2018-12-14,3564.931030,14185.304951,6.626195,20.439145,739.735614,185.168664,9.066509,71.238935,1829.082160
2018-12-15,3560.300126,14271.019359,6.653408,19.739442,745.852538,181.791786,8.646345,72.185682,1820.683803


In [57]:
yhat.to_csv('predictions.csv')